In [2]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt

pd.set_option('display.float_format', lambda x: '%.4f' % x)

# 2 Analyzing GMO

Examine GMO’s performance. Use the risk-free rate to convert the total returns to excess returns

In [3]:
df_gmo = pd.read_excel('/Users/peterfeng/Downloads/积累/Previous Courses Materials/UChicago Past Courses/UChicago 2021 Fall/Portfolio Theory/hw data/gmo_analysis_data.xlsx', sheet_name = 'returns (total)').set_index('Date')
df_risk_free = pd.read_excel('/Users/peterfeng/Downloads/积累/Previous Courses Materials/UChicago Past Courses/UChicago 2021 Fall/Portfolio Theory/hw data/gmo_analysis_data.xlsx', sheet_name = 'risk-free rate').set_index('Date').dropna()

In [4]:
df_gmo_excess = df_gmo.dropna().subtract(df_risk_free['US3M'][list(df_gmo.dropna().index)[0]:], axis = 0)

## 1. Calculate the mean, volatility, and Sharpe ratio for GMWAX. Do this for three samples: 

• from inception through 2011

• 2012-present

• inception - present

Has the mean, vol, and Sharpe changed much since the case?

In [6]:
stat_table_1 = pd.DataFrame(columns=['mean', 'vol', 'Sharpe'],
        index = ['incept-2011', '2012-present', 'all'])

stat_table_1.loc['incept-2011', 'mean'] = df_gmo_excess['GMWAX'][: '2011'].mean() * 12
stat_table_1.loc['incept-2011', 'vol'] = df_gmo_excess['GMWAX'][: '2011'].std() * np.sqrt(12)
stat_table_1.loc['incept-2011', 'Sharpe'] = stat_table_1.loc['incept-2011', 'mean'] / stat_table_1.loc['incept-2011', 'vol']

stat_table_1.loc['2012-present', 'mean'] = df_gmo_excess['GMWAX']['2012': ].mean() * 12
stat_table_1.loc['2012-present', 'vol'] = df_gmo_excess['GMWAX']['2012': ].std() * np.sqrt(12)
stat_table_1.loc['2012-present', 'Sharpe'] = stat_table_1.loc['2012-present', 'mean'] / stat_table_1.loc['2012-present', 'vol']

stat_table_1.loc['all', 'mean'] = df_gmo_excess['GMWAX'].mean() * 12
stat_table_1.loc['all', 'vol'] = df_gmo_excess['GMWAX'].std() * np.sqrt(12)
stat_table_1.loc['all', 'Sharpe'] = stat_table_1.loc['all', 'mean'] / stat_table_1.loc['all', 'vol']

stat_table_1

,mean,vol,Sharpe
incept-2011,0.0158,0.1250,0.1266
2012-present,0.0593,0.0853,0.6952
all,0.0329,0.1111,0.2964


The mean and sharpe ratio increase since the case, while the volatility drops. Since the case, all the stats have become better. Mean increased to about 5 times, vol droped 30%, and sharpe ratio has an increase for more than 5 times. 

## 2. GMO believes a risk premium is compensation for a security’s tendency to lose money at “bad times”. 
For all three samples, analyze extreme scenarios by looking at

• Min return

• 5th percentile (VaR-5th) 

• Maximum drawdown

In [7]:
def max_drawdown(df):
    df_compound = (df + 1).cumprod()
    df_currmax = df_compound.cummax()
    df_max_drawdown_day = (df_compound - df_currmax) / df_currmax
    return df_max_drawdown_day.min()

In [8]:
stat_table_2 = pd.DataFrame(columns=['min', 'VaR', 'max-drawdown'],
        index = ['incept-2011', '2012-present', 'all'])

stat_table_2.loc['incept-2011', 'min'] = df_gmo_excess['GMWAX'][: '2011'].min()
stat_table_2.loc['incept-2011', 'VaR'] = df_gmo_excess['GMWAX'][: '2011'].quantile(.05)
stat_table_2.loc['incept-2011', 'max-drawdown'] = max_drawdown(df_gmo['GMWAX'][: '2011'])

stat_table_2.loc['2012-present', 'min'] = df_gmo_excess['GMWAX']['2012': ].min() 
stat_table_2.loc['2012-present', 'VaR'] = df_gmo_excess['GMWAX']['2012': ].quantile(.05)
stat_table_2.loc['2012-present', 'max-drawdown'] = max_drawdown(df_gmo['GMWAX']['2012': ])

stat_table_2.loc['all', 'min'] = df_gmo_excess['GMWAX'].min() 
stat_table_2.loc['all', 'VaR'] = df_gmo_excess['GMWAX'].quantile(.05) 
stat_table_2.loc['all', 'max-drawdown'] = max_drawdown(df_gmo['GMWAX'])

stat_table_2

,min,VaR,max-drawdown
incept-2011,-0.1492,-0.0598,-0.3552
2012-present,-0.1187,-0.0306,-0.1675
all,-0.1492,-0.0449,-0.3552


### (a) Does GMWAX have high or low tail-risk as seen by these stats? 

It has quite high tail risk given the statistics, where VaR and max-drawdown are much larger than the mean return in all periods. Looking at the entire historical data, it seems GMWAX has a high tail-risk. But if only look at the data since the case, its tail-risk decreases greatly. 

### (b) Does that vary much across the two subsamples?

The Var and max-drawdown vary much in the two subsamples, with the 2012-after sample having much lower tail risk than the previous sample. 

## 3. For all three samples, regress excess returns of GMWAX on excess returns of SPY.
### (a) Report the estimated alpha, beta, and r-squared.

In [7]:
stat_table_3 = pd.DataFrame(columns=['alpha', 'beta', 'r-squared'],
        index = ['incept-2011', '2012-present', 'all'])

X = sm.add_constant(df_gmo_excess['SPY'][:'2011'])
y = df_gmo_excess['GMWAX'][:'2011']
model = sm.OLS(y, X).fit()

stat_table_3.loc['incept-2011', 'alpha'] = model.params[0]
stat_table_3.loc['incept-2011', 'beta'] = model.params[1]
stat_table_3.loc['incept-2011', 'r-squared'] = model.rsquared

X = sm.add_constant(df_gmo_excess['SPY']['2012':])
y = df_gmo_excess['GMWAX']['2012':]
model = sm.OLS(y, X).fit()

stat_table_3.loc['2012-present', 'alpha'] = model.params[0]
stat_table_3.loc['2012-present', 'beta'] = model.params[1]
stat_table_3.loc['2012-present', 'r-squared'] = model.rsquared

X = sm.add_constant(df_gmo_excess['SPY'])
y = df_gmo_excess['GMWAX']
model = sm.OLS(y, X).fit()

stat_table_3.loc['all', 'alpha'] = model.params[0]
stat_table_3.loc['all', 'beta'] = model.params[1]
stat_table_3.loc['all', 'r-squared'] = model.rsquared

stat_table_3


/opt/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


,alpha,beta,r-squared
incept-2011,-0.0005,0.5396,0.5071
2012-present,-0.0024,0.5683,0.7633
all,-0.0011,0.5461,0.5668


### (b) Is GMWAX a low-beta strategy? Has that changed since the case?

Yes, GMWAX is a low-beta strategy since it has beta constantly lower than 1. And since the case it remained almostly unchanged. 

### (c) Does GMWAX provide alpha? Has that changed across the subsamples?

It does not provide alpha since the alpha is really small as seen from the above table. It also does not change much across the samples, although it increased a little since the case.

# 3 Forecast Regressions

## 1. Consider the lagged regression, where the regressor, (X, ) is a period behind the target, ($ r^{SPY}$ ).

$$ r^{SPY}_t = \alpha^{SPY,X} + ( \beta^{SPY,X})′ X_{t-1} + \epsilon^{SPY,X}_t $$

Estimate and report the R^2, as well as the OLS estimates for $\alpha$ and $\beta$. Do this for...

• X as a single regressor, the dividend-price ratio.

• X as a single regressor, the earnings-price ratio.

• X as three regressors, the dividend-price ratio, the earnings-price ratio, and the 10-year yield.

For each, report the r-squared.


** See tables below

## 2. For each of the three regressions, let’s try to utilize the resulting forecast in a trading strategy.

Build the forecasted SPY returns: $ \hat{r}^{SPY}_{t+1} $  . Note that this denotes the forecast made using $X_t$ to forecast the (t + 1) return.

Set the scale of the investment in SPY equal to 100 times the forecasted value:

$$ \omega_t = 100 \hat{r}^{SPY}_{t+1} $$

We are not taking this scaling too seriously. We just want the strategy to go bigger in periods where the forecast is high and to withdraw in periods where the forecast is low, or even negative.

Calculate the return on this strategy: 

$$ r^X_{t+1} = \omega_t r^{SPY}_{t+1}$$

For each strategy, estimate 

• mean, volatility, Sharpe, 

• max-drawdown

• market alpha

• market beta

• market Information ratio

** See tables below

In [8]:
df_signals = pd.read_excel('gmo_analysis_data.xlsx', sheet_name = 'signals').set_index('Date').dropna()
df_gmo_excess = df_gmo.subtract(df_risk_free['US3M'], axis = 0)

In [9]:
ep_predict = pd.DataFrame(columns = ['predicted', 'r-squared', 'alpha', 'beta'], index = df_gmo_excess['SPY'][1:].index)
all_stats_table = pd.DataFrame(columns = ['mean', 'vol', 'Sharpe', 
                                   'market alpha', 'market beta', 'information ratio', 'max drawdown'], 
                        index = ['ep_table', 'dp_table', 'all_table'])


for i in range(3, df_gmo_excess['SPY'][1:].shape[0]):
    X = sm.add_constant(df_signals['EP'].iloc[:i].shift(1))
    y = df_gmo_excess['SPY'].iloc[: i]
    model_ep = sm.OLS(y, X, missing = 'drop').fit()
    ep_predict.iloc[i]['r-squared'] = model_ep.rsquared
    predicted = model_ep.predict(np.asarray([1, df_signals['EP'][i]], dtype = float)) * df_gmo_excess['SPY'].iloc[i] * 100
    ep_predict.iloc[i]['predicted'] = predicted[0]
    ep_predict['alpha'].iloc[i] = model_ep.params[0]
    ep_predict['beta'].iloc[i] = model_ep.params[1]

display(ep_predict)
    
predictions = ep_predict['Dec 1996':'Oct 2020']['predicted'].dropna()
all_stats_table.loc['ep_table', 'mean'] = predictions.mean() * 12
all_stats_table.loc['ep_table', 'vol'] = predictions.std() * np.sqrt(12)
all_stats_table.loc['ep_table', 'Sharpe'] = all_stats_table.loc['ep_table', 'mean'] / all_stats_table.loc['ep_table', 'vol']
market_model = sm.OLS(np.asarray(df_gmo_excess['SPY'][predictions.index], dtype = float), 
                      sm.add_constant(np.asarray(predictions, dtype = float))).fit()
all_stats_table.loc['ep_table', 'market alpha'] = market_model.params[0]
all_stats_table.loc['ep_table', 'market beta'] = market_model.params[1]
all_stats_table.loc['ep_table', 'information ratio'] = market_model.params[0] / market_model.resid.std()
all_stats_table.loc['ep_table', 'max drawdown'] = max_drawdown(predictions)

,predicted,r-squared,alpha,beta
Date,,,,
1993-03-31,NaN,NaN,NaN,NaN
1993-04-30,NaN,NaN,NaN,NaN
1993-05-31,NaN,NaN,NaN,NaN
1993-06-30,-0.1853,1.0000,-7.0860,1.6004
1993-07-31,-0.0212,0.9942,-7.4103,1.6740
...,...,...,...,...
2021-06-30,0.0027,0.0083,-0.0069,0.0031
2021-07-31,-0.0002,0.0082,-0.0067,0.0030
2021-08-31,0.0108,0.0081,-0.0065,0.0030


In [10]:
dp_predict = pd.DataFrame(columns = ['predicted', 'r-squared','alpha','beta'], index = df_gmo_excess['SPY'][1:].index)

for i in range(3, df_gmo_excess['SPY'][1:].shape[0]):
    X = sm.add_constant(df_signals['DP'].iloc[:i].shift(1))
    y = df_gmo_excess['SPY'].iloc[:i]
    model_dp = sm.OLS(y, X, missing = 'drop').fit()
    dp_predict.iloc[i]['r-squared'] = model_dp.rsquared
    predicted = model_dp.predict(np.asarray([1, df_signals['DP'].iloc[i]])) * df_gmo_excess['SPY'].iloc[i] * 100
    dp_predict.iloc[i]['predicted'] = predicted[0]
    dp_predict.iloc[i]['alpha'] = model_dp.params[0]
    dp_predict.iloc[i]['beta'] = model_dp.params[1]
    
display(dp_predict)

predictions = dp_predict['predicted'].dropna()
all_stats_table.loc['dp_table', 'mean'] = predictions.mean() * 12
all_stats_table.loc['dp_table', 'vol'] = predictions.std() * np.sqrt(12)
all_stats_table.loc['dp_table', 'Sharpe'] = all_stats_table.loc['dp_table', 'mean'] / all_stats_table.loc['dp_table', 'vol']
market_model = sm.OLS(np.asarray(df_gmo_excess['SPY'][predictions.index], dtype = float), 
                      sm.add_constant(np.asarray(predictions, dtype = float))).fit()
all_stats_table.loc['dp_table', 'market alpha'] = market_model.params[0]
all_stats_table.loc['dp_table', 'market beta'] = market_model.params[1]
all_stats_table.loc['dp_table', 'information ratio'] = market_model.params[0] / market_model.resid.std()
all_stats_table.loc['dp_table', 'max drawdown'] = max_drawdown(predictions)

,predicted,r-squared,alpha,beta
Date,,,,
1993-03-31,NaN,NaN,NaN,NaN
1993-04-30,NaN,NaN,NaN,NaN
1993-05-31,NaN,NaN,NaN,NaN
1993-06-30,0.0252,1.0000,-2.6880,0.9603
1993-07-31,-0.0009,0.9942,-2.8102,1.0044
...,...,...,...,...
2021-06-30,0.0013,0.0095,-0.0110,0.0094
2021-07-31,0.0043,0.0095,-0.0109,0.0094
2021-08-31,0.0043,0.0092,-0.0105,0.0092


In [11]:
all_predict = pd.DataFrame(columns = ['predicted', 'r-squared','alpha','beta'], index = df_gmo_excess['SPY'][1:].index)

for i in range(3, df_gmo_excess['SPY'][1:].shape[0]):
    X = sm.add_constant(df_signals.iloc[: i], prepend = True, has_constant = 'add')
    y = df_gmo_excess['SPY'].iloc[:i]
    model_all = sm.OLS(np.asarray(y, dtype = float), np.asarray(X, dtype = float), missing = 'drop').fit()
    all_predict.iloc[i]['r-squared'] = model_all.rsquared
    predicted = ((model_all.params[1:] * df_signals.iloc[i]).sum() + model_all.params[0]) * 100 * df_gmo_excess['SPY'].iloc[i + 1]
    all_predict.iloc[i]['predicted'] = predicted
    all_predict.iloc[i]['alpha'] = model_all.params[0]
    all_predict.iloc[i]['beta'] = model_all.params[1]

display(all_predict) 
    
predictions = all_predict['predicted'].dropna()
all_stats_table.loc['all_table', 'mean'] = predictions.mean() * 12
all_stats_table.loc['all_table', 'vol'] = predictions.std() * np.sqrt(12)
all_stats_table.loc['all_table', 'Sharpe'] = all_stats_table.loc['all_table', 'mean'] / all_stats_table.loc['all_table', 'vol']
market_model = sm.OLS(np.asarray(df_gmo_excess['SPY'][predictions.index], dtype = float), 
                      sm.add_constant(np.asarray(predictions, dtype = float))).fit()
all_stats_table.loc['all_table', 'market alpha'] = market_model.params[0]
all_stats_table.loc['all_table', 'market beta'] = market_model.params[1]
all_stats_table.loc['all_table', 'information ratio'] = market_model.params[0] / market_model.resid.std()
all_stats_table.loc['all_table', 'max drawdown'] = max_drawdown(predictions)

display(all_stats_table)

,predicted,r-squared,alpha,beta
Date,,,,
1993-03-31,NaN,NaN,NaN,NaN
1993-04-30,NaN,NaN,NaN,NaN
1993-05-31,NaN,NaN,NaN,NaN
1993-06-30,-0.0452,1.0000,1.8131,-2.3715
1993-07-31,-0.7058,1.0000,24.2807,2.5341
...,...,...,...,...
2021-06-30,0.0235,0.0050,0.0115,-0.0027
2021-07-31,0.0214,0.0050,0.0114,-0.0027
2021-08-31,0.0337,0.0050,0.0120,-0.0028


,mean,vol,Sharpe,market alpha,market beta,information ratio,max drawdown
ep_table,0.0387,0.1468,0.2636,0.0064,-0.1498,0.1461,-0.6090
dp_table,0.0304,0.1529,0.1987,0.0074,-0.0043,0.1746,-0.6377
all_table,0.0493,0.2956,0.1666,0.0071,0.0756,0.1689,-0.9097


## 3. GMO believes a risk premium is compensation for a security’s tendency to lose money at “bad times”. Let’s consider risk characteristics.

### (a) For both strategies, the market, and GMO, calculate the monthly VaR for π = .05. Just use the quantile of the historic data for this VaR calculation.

In [12]:
print('SPY VaR', df_gmo_excess['SPY'].quantile(.05))
print('GMO VaR', df_gmo_excess['GMWAX'].dropna().quantile(.05))

print('EP VaR', ep_predict['predicted'].dropna().quantile(.05))
print('DP VaR', dp_predict['predicted'].dropna().quantile(.05))

print('All VaR', all_predict['predicted'].dropna().quantile(.05))

SPY VaR -0.06958505455369124
GMO VaR -0.044914532965437855
EP VaR -0.055111833318239246
DP VaR -0.06918605553394627
All VaR -0.06336452493693702


### (b) The GMO case mentions that stocks under-performed short-term bonds from 2000-2011. Does the dynamic portfolio above under-perform the risk-free rate over this time?

In [13]:
print('SPY Mean', df_gmo_excess['SPY']['2000': '2011'].mean() * 12)
print('GMO Mean', df_gmo_excess['GMWAX']['2000': '2011'].dropna().mean() * 12)

print('EP Mean', 12 * (ep_predict['predicted']['2000': '2011'].mean() - df_risk_free['US3M']['2000': '2011'].mean())) 
print('DP Mean', 12 * (dp_predict['predicted']['2000': '2011'].mean() - df_risk_free['US3M']['2000': '2011'].mean()))

print('All Mean', 12 * (all_predict['predicted']['2000': '2011'].mean() - df_risk_free['US3M']['2000': '2011'].mean()))

SPY Mean -0.0049036843992595214
GMO Mean 0.03306084278085366
EP Mean -0.017868121178441124
DP Mean -0.07068101842017308
All Mean -0.03051203921606162


These strategies actually outperforms the risk-free rate given that the market is down. 

### (c) Based on the regression estimates, in how many periods do we estimate a negative risk premium?

In [14]:
(dp_predict['alpha'] + dp_predict['beta'] * df_signals.DP).apply(lambda x: 1 if x < 0 else 0).sum()

15

In [15]:
(ep_predict['alpha'] + ep_predict['beta'] * df_signals.EP).apply(lambda x: 1 if x < 0 else 0).sum()

55

The dynamic portfolio would predict some negative period over time, with EP prediction having more than DP prediction. 

### (d) Do you believe the dynamic strategy takes on extra risk??

In [16]:
stat_table_4 = pd.DataFrame(columns=['mean', 'vol', 'Sharpe'],
        index = ['incept-2011', '2012-present', 'all'])

stat_table_4.loc['incept-2011', 'mean'] = df_gmo_excess['SPY'][: '2011'].mean() * 12
stat_table_4.loc['incept-2011', 'vol'] = df_gmo_excess['SPY'][: '2011'].std() * np.sqrt(12)
stat_table_4.loc['incept-2011', 'Sharpe'] = stat_table_4.loc['incept-2011', 'mean'] / stat_table_4.loc['incept-2011', 'vol']

stat_table_4.loc['2012-present', 'mean'] = df_gmo_excess['SPY']['2012': ].mean() * 12
stat_table_4.loc['2012-present', 'vol'] = df_gmo_excess['SPY']['2012': ].std() * np.sqrt(12)
stat_table_4.loc['2012-present', 'Sharpe'] = stat_table_4.loc['2012-present', 'mean'] / stat_table_4.loc['2012-present', 'vol']

stat_table_4.loc['all', 'mean'] = df_gmo_excess['SPY'].mean() * 12
stat_table_4.loc['all', 'vol'] = df_gmo_excess['SPY'].std() * np.sqrt(12)
stat_table_4.loc['all', 'Sharpe'] = stat_table_4.loc['all', 'mean'] / stat_table_4.loc['all', 'vol']

stat_table_4

,mean,vol,Sharpe
incept-2011,0.0538,0.1526,0.3528
2012-present,0.1545,0.1311,1.1785
all,0.0883,0.1461,0.6044


I think that the dynamic strategy doesn't take on extra risk, as the volatility of GWMAX is lower than the market volatility in all three periods. 

# 4 Out-of-Sample Forecasting

## 1. Report the out-of-sample R2 :

$$ R^2_{OOS} = 1 - \frac{\sum^T_{i=61} (e^x_i)^2}{\sum^T_{i=61} (e^0_i)^2}$$

Note that unlike an in-sample r-squared, the out-of-sample r-squared can be anywhere between
(−∞, 1].

Did this forecasting strategy produce a positive OOS r-squared?

In [17]:
oos_table = pd.DataFrame(columns = ['beta', 'alpha', 'error', 'null error', 'predicted'], index = df_gmo['SPY'].index)
for i in range(60, df_gmo['SPY'].shape[0] - 1):
    X = sm.add_constant(df_signals['EP'].iloc[: i - 1])
    y = df_gmo_excess['SPY'].iloc[1: i]
    model = sm.OLS(np.asarray(y), 
                   np.asarray(X)).fit()
    predicted = model.predict([1, df_signals['EP'].iloc[i]])[0]
    error = predicted - df_gmo_excess['SPY'].iloc[i]
    oos_table['predicted'].iloc[i] = predicted * 100 * df_gmo_excess['SPY'].iloc[i]
    oos_table['error'].iloc[i] = error
    r_bar = y.mean()
    null_error = df_gmo_excess['SPY'].iloc[i] - r_bar
    oos_table['null error'].iloc[i] = null_error
    oos_table['alpha'].iloc[i] = model.params[0]
    oos_table['beta'].iloc[i] = model.params[1]

    
oos_table.dropna()

,beta,alpha,error,null error,predicted
Date,,,,,
1998-02-28,0.0067,-0.0235,-0.0623,0.0529,0.0164
1998-03-31,0.0047,-0.0121,-0.0391,0.0316,0.0237
1998-04-30,0.0033,-0.0040,-0.0009,-0.0047,0.0067
1998-05-31,0.0033,-0.0039,0.0327,-0.0382,-0.0194
1998-06-30,0.0046,-0.0111,-0.0333,0.0257,0.0194
...,...,...,...,...,...
2021-05-31,0.0031,-0.0069,-0.0025,-0.0006,0.0027
2021-06-30,0.0030,-0.0067,-0.0225,0.0153,-0.0002
2021-07-31,0.0030,-0.0065,-0.0199,0.0171,0.0108


In [18]:
oos_rsquared = 1 - (oos_table['error'] ** 2).mean() / (oos_table['null error'] ** 2).mean()
oos_rsquared

-0.023613091904505

No, it doens't produce a positive r-squared. 

## 2. Re-do problem 3.2 using this OOS forecast. How much better/worse is the OOS Earnings-Price ratio strategy compared to the in-sample version of 3.2?

In [19]:
all_stats_table_1 = pd.DataFrame(columns = ['mean', 'vol', 'Sharpe', 
                                   'market alpha', 'market beta', 'information ratio', 'max drawdown'], 
                        index = ['oos_table'])

predictions = oos_table['predicted'].dropna()

all_stats_table_1.loc['oos_table', 'mean'] = predictions.mean() * 12
all_stats_table_1.loc['oos_table', 'vol'] = predictions.std() * np.sqrt(12)
all_stats_table_1.loc['oos_table', 'Sharpe'] = all_stats_table_1.loc['oos_table', 'mean'] / all_stats_table_1.loc['oos_table', 'vol']
market_model = sm.OLS(np.asarray(df_gmo_excess['SPY'][predictions.index], dtype = float), 
                      sm.add_constant(np.asarray(predictions, dtype = float))).fit()
all_stats_table_1.loc['oos_table', 'market alpha'] = market_model.params[0]
all_stats_table_1.loc['oos_table', 'market beta'] = market_model.params[1]
all_stats_table_1.loc['oos_table', 'information ratio'] = market_model.params[0] / market_model.resid.std()
all_stats_table_1.loc['oos_table', 'max drawdown'] = max_drawdown(predictions)

display(all_stats_table_1)

,mean,vol,Sharpe,market alpha,market beta,information ratio,max drawdown
oos_table,0.0335,0.1461,0.2295,0.0055,0.2402,0.1285,-0.6090


In [20]:
(all_stats_table_1.loc['oos_table', :] - all_stats_table.loc['ep_table', :]) / all_stats_table.loc['ep_table', :]

mean                -0.1332
vol                 -0.0048
Sharpe              -0.1290
market alpha        -0.1345
market beta         -2.6033
information ratio   -0.1203
max drawdown        -0.0000
dtype: object

It doesn't perform too much worse compared to the in-sample forecast. 

## 3. Re-do problem 3.3 using this OOS forecast. Is the point-in-time version of the strategy riskier?

In [21]:
predictions.dropna().quantile(.05)

-0.055021987103332

In [22]:
12 * (predictions['2000': '2011'] -  df_risk_free['US3M']['2000': '2011']).mean()

-0.02056525585751419

No, it is not riskier. Both the VaR and the risk premium during the loss period are smaller. 